In [ ]:
import os

import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv

from datautil.pipeline import load_players_and_teams, insert_fixture_records
from datautil.updating.pipeline import update_local_data
from datautil.utilities import get_current_season, get_next_gameweek, get_previous_seasons
from api.fpl import get_fixture_data, get_bootstrap_data
from features.features import engineer_features
from predictions import make_predictions, group_predictions_by_gameweek, weight_gameweek_predictions_by_availability
from predictions import PositionSplitEstimator
from simulation.utilities import get_player_name
from optimize import optimize_squad
from optimize.utilities import suggest_squad_roles

In [ ]:
load_dotenv()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
def authenticate() -> requests.Session:
    """Logs into FPL and returns an authenticated session."""

    email = os.getenv('FPL_EMAIL')
    password = os.getenv('FPL_PASSWORD')
    cookie = os.getenv('FPL_COOKIE')

    session = requests.Session()

    url = 'https://users.premierleague.com/accounts/login/'
    headers = {
        'User-Agent': 'Dalvik/2.1.0 (Linux; U; Android 5.1; PRO 5 Build/LMY47D)',
        'Cookie': cookie
    }
    payload = {
        'login': email,
        'password': password,
        'app': 'plfpl-web',
        'redirect_uri': 'https://fantasy.premierleague.com/a/login',
    }

    response = session.post(url, data=payload, headers=headers)
    if not response.ok:
        raise Exception(f'Failed to log in! Reason: {response.reason}')
    
    return session


def get_my_team_data() -> dict:
    """Fetches current squad selection from FPL."""

    session = authenticate()

    manager_data = session.get('https://fantasy.premierleague.com/api/me/')
    manager_id = manager_data.json()['player']['entry']

    team_data = session.get(f'https://fantasy.premierleague.com/api/my-team/{manager_id}/')
    team_data = team_data.json()

    return team_data


def print_transfer_summary(
        initial_squad: set, best_squad: set, best_squad_roles: dict, 
        bootstrap_elements: pd.DataFrame, next_gameweek_predictions: pd.Series
    ):
    """Print a user-friendly summary of transfers from an initial squad."""

    print("Starting XI")
    for element in best_squad_roles['starting_xi']:
        name = get_player_name(element, bootstrap_elements)
        predicted_points = next_gameweek_predictions[element]
        print(f"{name} ({predicted_points:.2f})")

    print("\nReserves")
    for element in [best_squad_roles['reserve_gkp'], *best_squad_roles['reserve_out']]:
        name = get_player_name(element, bootstrap_elements)
        predicted_points = next_gameweek_predictions[element]
        print(f"{name} ({predicted_points:.2f})")

    captain = get_player_name(best_squad_roles['captain'], bootstrap_elements)
    vice_captain = get_player_name(best_squad_roles['vice_captain'], bootstrap_elements)
    print(f"\nCaptain: {captain}")
    print(f"Vice Captain: {vice_captain}")
    
    print("\nTransfers in")
    for element in best_squad - initial_squad:
        name = get_player_name(element, bootstrap_elements)
        predicted_points = next_gameweek_predictions[element]
        print(f"{name} ({predicted_points:.2f})")

    print("\nTransfers out")
    for element in initial_squad - best_squad:
        name = get_player_name(element, bootstrap_elements)
        predicted_points = next_gameweek_predictions[element]
        print(f"{name} ({predicted_points:.2f})")

In [ ]:
# Load API data
fixtures = pd.DataFrame(get_fixture_data())
fixtures['kickoff_time'] = pd.to_datetime(fixtures['kickoff_time'])
bootstrap = get_bootstrap_data()
bootstrap_events = pd.DataFrame(bootstrap['events'])
bootstrap_events['deadline_time'] = pd.to_datetime(bootstrap_events['deadline_time'])
bootstrap_elements = pd.DataFrame(bootstrap['elements'])
bootstrap_elements.set_index('id', inplace=True, drop=False)
bootstrap_elements['chance_of_playing_next_round'].fillna(100, inplace=True)
bootstrap_teams = pd.DataFrame(bootstrap['teams'])

In [ ]:
current_season = get_current_season(bootstrap_events)
seasons = get_previous_seasons(current_season)
next_gameweek = get_next_gameweek(bootstrap_events)

In [ ]:
update_local_data(current_season, bootstrap_elements, bootstrap_events, bootstrap_teams)
local_players, local_teams = load_players_and_teams(seasons)
local_players, local_teams = insert_fixture_records(current_season, next_gameweek, fixtures, local_players, local_teams, bootstrap_elements, bootstrap_teams)

In [ ]:
features, columns = engineer_features(local_players, local_teams)
# Keep only features for the current season
features = features[(features['season'] == current_season)]

In [ ]:
model_path = f"models/2024-25/model-all.pkl"
columns_path = f"models/2024-25/columns.json"
predictions = make_predictions(features, model_path, columns_path)
gameweek_predictions = group_predictions_by_gameweek(predictions)
gameweek_predictions = weight_gameweek_predictions_by_availability(gameweek_predictions, bootstrap_elements, next_gameweek)

In [ ]:
# Load the FPL team information
my_team = get_my_team_data()
initial_picks = pd.DataFrame(my_team['picks'])
initial_budget_remaining = my_team['transfers']['bank']
initial_squad = set(initial_picks['element'])
selling_prices = initial_picks.set_index('element')['selling_price']
transfer_limit = my_team['transfers']['limit']
transfers_made = my_team['transfers']['made']
free_transfers = float('inf') if transfer_limit is None else (transfer_limit - transfers_made)

In [ ]:
wildcard_gameweeks = [14, 25]
now_costs = bootstrap_elements['now_cost']
positions = bootstrap_elements['element_type']
next_gameweek_predictions = gameweek_predictions.loc[:, next_gameweek]

best_squad = optimize_squad(
    current_season, initial_squad, initial_budget_remaining, 
    next_gameweek, wildcard_gameweeks, 
    now_costs, selling_prices, 
    bootstrap_elements, gameweek_predictions
)
best_squad_roles = suggest_squad_roles(
    best_squad, positions.to_dict(), 
    next_gameweek_predictions.to_dict()
)

In [ ]:
print_transfer_summary(
    initial_squad, best_squad, best_squad_roles, 
    bootstrap_elements, next_gameweek_predictions
)